# Notebook used to retreive sentinel-1 SLC data from ASF DAAC

In [ ]:
import os
from collections import Counter
import asf_search as asf
import geemap

# Set parameters for search and storage
- edit to include flightDirection and perpendicular baseline?

In [ ]:
def project_dir(cwd, project_name):
    work_dir = os.path.join(cwd, project_name)
    os.makedirs(work_dir, exist_ok=True)
    data_dir = os.path.join(work_dir,'0_initial')
    os.makedirs(data_dir, exist_ok=True)

    return work_dir, data_dir

def SLC_search_results(startdate, enddate, wkt_aoi, pform, proclevel, bmode, flightD):
    results = asf.search(
        platform= pform,
        processingLevel=[proclevel],
        start = startdate,
        end = enddate,
        intersectsWith = wkt_aoi,
        beamMode= bmode,
        flightDirection= flightD
    )
    
    return results.geojson(), results

def SLC_download(earthdatausr, earthdatapass, directory, metadata, results, direction):
    #establish download session
    session = asf.ASFSession().auth_with_creds(earthdatausr, earthdatapass)

    #create empty dictionary to store epoch information
    epoch_info ={}

    # paths, frames, filenames, and asf search products into list
    paths = [feature["properties"]["pathNumber"] for feature in metadata['features']]
    frames = [feature["properties"]["frameNumber"] for feature in metadata['features']]
    filenames = [feature["properties"]["fileName"] for feature in metadata['features']]
    asfprods = list(results)

    # create directories for each path-frame combination, used to store like epochs (easy processing later)
    for path in list(Counter(paths)):
        path_dir = os.path.join(directory, f"{direction}/{path}")
        os.makedirs(path_dir, exist_ok=True)
        for frame in list(Counter(frames)):
            frame_dir = os.path.join(path_dir, f"{frame}")
            os.makedirs(frame_dir, exist_ok=True)

    # popualate epoch_info
    for name, path, frame, asfprod in zip(filenames, paths, frames, asfprods):
        epoch_info[name] = [path, frame, asfprod]

    # download SLC epochs from ASF vertex to apprroprate directories
    for name in epoch_info:
        path = epoch_info[name][0]
        frame = epoch_info[name][1]
        asfprod = epoch_info[name][2]

        asfprod.download(
            path = os.path.join(directory, f'{path}/{frame}'),
            session = session
        )

# Set working directory

In [ ]:
work_dir, data_dir = project_dir('/home/wcc/Desktop/SabineRS', 'Sentinel-1')

# Draw Area of Interest for your data search

In [ ]:
## interactive map for you to draw a polygon to signify your aoi

## Create a map centered at a specific location
m = geemap.Map(center=[20, 0], zoom=2, basemap='HYBRID')

## Add drawing tools
m.add_draw_control()

## Display the map
display(m)

In [ ]:
# get feature you just drew
draw_features = m.draw_features[0]

# extract the vertices pf the feature
vertices = [(coords[0], coords[1]) for i, coords in enumerate(draw_features.getInfo()['geometry']['coordinates'][0])]
vertices.reverse()

# create POLYGON string to use when searching asf for imagery
aoi = "POLYGON((" + ", ".join([f'{str(northing)[:-2]} {str(easting)[:-2]}' for northing, easting in vertices[:]]) +"))"

# Find SLC images for processing
- Backscatter and GLCM used for training classification model
- Phase will be used for subpixel reclassification of the subaqueous land class, resulting in three total classes

In [ ]:
asc_metadata, asc_results = SLC_search_results('2019-10-01', 
                                       '2024-11-25',
                                       aoi, 
                                       asf.PLATFORM.SENTINEL1A,
                                       asf.PRODUCT_TYPE.SLC,
                                       asf.BEAMMODE.IW,
                                       asf.FLIGHT_DIRECTION.ASCENDING
                                       )

if len(asc_metadata['features']) == 0:
    print('\nNo ascending track epochs')
else:
    print(f'\n{len(asc_metadata["features"])} ascending track epochs')

In [ ]:
des_metadata, des_results = SLC_search_results('2019-10-01', 
                                       '2024-11-25',
                                       aoi, 
                                       asf.PLATFORM.SENTINEL1A,
                                       asf.PRODUCT_TYPE.SLC,
                                       asf.BEAMMODE.IW,
                                       asf.FLIGHT_DIRECTION.DESCENDING
                                       )

if len(des_metadata['features']) == 0:
    print('\nNo descending track epochs')
else:
    print(f'\n{len(des_metadata["features"])} descending track epochs')

# Filter out images along the minority track
- only downloads images that are along the same track, easier interpretation
- remove this step if want all images

In [ ]:
direction = 'ASCENDING' # or DESCENDING
if direction == 'ASCENDING':
    directionmetadata = asc_metadata
    directionresults = asc_results
else:
    directionmetadata = des_metadata
    directionresults = des_results


# put in your NASA Earth Data login info below
SLC_download(USERNAME, PASSWORD, data_dir, directionmetadata, directionresults, direction)

# Move to sentinel1prep_snap notebook (not created just yet) for processing using ESA SNAP and snapista python wrapper